In [30]:
import sys,os
sys.path.append(os.path.realpath('..'))

from scripts.functions import dilute, replenish,dilute_species, replenish_species, run_model,plot_result
from scripts.species import DNA, RNA, Protein, Resource
from scripts.models import Repressor_model


import numpy as np
import scipy.integrate
import scipy.optimize
from bokeh.plotting import figure, output_file, show

import bokeh.plotting
import bokeh.io
from bokeh.models import Span


# We will use Matplotlib to make a 3D plot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import panel as pn
pn.extension()

bokeh.io.output_notebook()

Loading BokehJS ...

In [86]:
#######Define species #######

#T7 RNAP
T7_DNA = DNA()
T7_RNA = RNA()
T7 = Protein()

#Repressor
Repressor_DNA = DNA()
Repressor_RNA = RNA()
Repressor = Protein()

#GFP
GFP_DNA = DNA()
GFP_RNA = RNA()
GFP = Protein()

#Resource
R      = Resource()

#Others
TX = 1
TL = 1

#list of molecules
molecules_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,Repressor_DNA,Repressor_RNA,Repressor,R]


#give index num
for i in range(0,len(molecules_list)):
    molecules_list[i].idx = i




#define values 
##T7
T7_DNA.ic =0.01
T7_DNA.lc =0.01

T7.ic = 1
T7.lc = 1

##GFP
GFP_DNA.ic = 0.01
GFP_DNA.lc = 0.01

GFP.lc = 0

##Repressor 
Repressor_DNA.ic = 0.1
Repressor_DNA.lc = 0.1


#resources
R.ic =100
R.lc =100

#Define parameters
alpha=0.7;
beta=0.07;
K=1.0;
Kd = 0.5;
HC = 1;
KTL=10;
KTX=1;
lamb1=0;
lamb2=0;

parameters_list = (alpha,beta,K,Kd,KTL,KTX,lamb1,lamb2)



In [87]:
#Positive control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]

result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(97,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).pc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(GFP.pc_result)

In [88]:
#Negative control
T7_DNA.ic = 0
T7_DNA.lc = 0

#initial value
molecules_0 = [None]*len(molecules_list) #create empty list


for i in range(0,len(molecules_list)):
    molecules_0[i] = molecules_list[i].ic

#stage 1 kick start
result_all=[molecules_0] #initial condition


#kick start
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [T7,R]
result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


#self regeneration 
dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
replenish_list = [R]
result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



#wash out
dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
replenish_list = [R]

result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



result_all = np.resize(result_all,(97,10))
for i in range(0,len(molecules_list)):
        (molecules_list[i]).nc_result = result_all.transpose()[i]

#T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
plot_result(T7.nc_result)

In [91]:
stored_data = []
for conc in np.logspace(-5,0, 10, endpoint=True):
    #experiment 
    T7_DNA.ic = conc
    T7_DNA.lc = conc
    #initial value
    molecules_0 = [None]*len(molecules_list) #create empty list

    for i in range(0,len(molecules_list)):
        molecules_0[i] = molecules_list[i].ic

    #stage 1 kick start
    result_all=[molecules_0] #initial condition


    #DR = 0.2

    #kick start
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [T7,R]
    result_all,molecules_0 = run_model(Repressor_model,(0,4),parameters_list,molecules_0,dilution_list,replenish_list,result_all)


    #self regeneration 
    dilution_list = [T7_RNA,T7,GFP,GFP_RNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]
    result_all,molecules_0 = run_model(Repressor_model,(4,16),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



    #wash out
    dilution_list = [T7_DNA,T7_RNA,T7,GFP_DNA,GFP,GFP_RNA,Repressor_DNA,Repressor,Repressor_RNA,R]
    replenish_list = [R]

    result_all,molecules_0=run_model(Repressor_model,(16,24),parameters_list,molecules_0,dilution_list,replenish_list,result_all)



    result_all = np.resize(result_all,(97,10))
    for i in range(0,len(molecules_list)):
            (molecules_list[i]).result = result_all.transpose()[i]

    #T7_DNA,T7_RNA,T7,GFP_DNA,GFP_RNA,GFP,R = result_all.transpose()
    plot_result(GFP.result)
    #print((GFP).result[60]/GFP.pc_result[60])
    stored_data.append((GFP).result[60]/GFP.pc_result[60])



In [90]:
conc = np.logspace(-4,1, 100, endpoint=True)

p = figure(width=800, height=400,x_axis_type = "log")
p.dot(conc,stored_data,size=20)
show(p)

In [46]:
print(conc)

[1.00000000e-05 1.12332403e-05 1.26185688e-05 1.41747416e-05
 1.59228279e-05 1.78864953e-05 2.00923300e-05 2.25701972e-05
 2.53536449e-05 2.84803587e-05 3.19926714e-05 3.59381366e-05
 4.03701726e-05 4.53487851e-05 5.09413801e-05 5.72236766e-05
 6.42807312e-05 7.22080902e-05 8.11130831e-05 9.11162756e-05
 1.02353102e-04 1.14975700e-04 1.29154967e-04 1.45082878e-04
 1.62975083e-04 1.83073828e-04 2.05651231e-04 2.31012970e-04
 2.59502421e-04 2.91505306e-04 3.27454916e-04 3.67837977e-04
 4.13201240e-04 4.64158883e-04 5.21400829e-04 5.85702082e-04
 6.57933225e-04 7.39072203e-04 8.30217568e-04 9.32603347e-04
 1.04761575e-03 1.17681195e-03 1.32194115e-03 1.48496826e-03
 1.66810054e-03 1.87381742e-03 2.10490414e-03 2.36448941e-03
 2.65608778e-03 2.98364724e-03 3.35160265e-03 3.76493581e-03
 4.22924287e-03 4.75081016e-03 5.33669923e-03 5.99484250e-03
 6.73415066e-03 7.56463328e-03 8.49753436e-03 9.54548457e-03
 1.07226722e-02 1.20450354e-02 1.35304777e-02 1.51991108e-02
 1.70735265e-02 1.917910